# F1 Race Simulation Notebook

This notebook test the `Run` class and demonstrates how to simulate races on the 2016 F1 timing database with both deterministic and stochastic modes.

## Configuration

In [1]:
import os
import sys

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Project root added to sys.path:", project_root)

Project root added to sys.path: c:\Users\mateo\Desktop\MCEM\new\Nouveau dossier\Simulation_Project


In [3]:
# === General configuration ===
season = 2016
location="Austin"
db_path = "../data/F1_timingdata_2014_2019.sqlite"

# === List of test GPs with associated strategy files for a given season ===

gp_configs = {
    "Austin": {
        "strategy_file": "strategies_austin_2016.pkl",
        "var_name": "simulator_austin"
    },
    "Suzuka": {
        "strategy_file": "strategies_suzuka_2016.pkl",
        "var_name": "simulator_suzuka"
    },
    "MexicoCity": {
        "strategy_file": "strategies_mexico_2016.pkl",
        "var_name": "simulator_mexico"
    },
    "YasMarina": {
        "strategy_file": "strategies_yasmarina_2016.pkl",
        "var_name": "simulator_yasmarina"
    }
}


## Load and Preprocess Data

In [4]:
from data_loader import DataLoader
from preprocessor import DataPreprocessor

# Load raw data
loader = DataLoader(db_path)
raw_data = loader.load_data()

required_columns = {
    "drivers": [
        "id",
        "name",
        "initials",
    ],
    "races": [
        "id",
        "season",
        "location",
        "nolapsplanned",
    ],
    "starterfields": [
        "race_id",
        "driver_id",
        "team",
        "status",
        "resultposition",
    ],
    "laps": [
        "race_id",
        "driver_id",
        "lapno",
        "laptime",
        "racetime",
        "compound",
        "tireage",
        "pitintime",
        "pitstopduration",
    ],
    "qualifyings": [
        "race_id",
        "driver_id",
        "position",
        "q1laptime",
        "q2laptime",
        "q3laptime",
    ],
    "retirements": [
        "season",
        "driver_id",
        "accidents",
        "failures",
    ],
    "fcyphases": [
        "race_id",
        "startlap",
        "endlap",
    ],
}



preprocessor = DataPreprocessor(required_columns)
data_tables = preprocessor.preprocess(raw_data)


## Run Stochastic Simulations

In [7]:
import pickle
from run import Run

# === Dictionary to store simulators by name ===
Probabilistic_simulators = {}

# === Loop through each GP ===
for gp, cfg in gp_configs.items():
    print("=" * 60)
    print(f" Running Simulation for: {gp}")

    # Load strategy
    with open(f"../data/{cfg['strategy_file']}", "rb") as f:
        driver_strategies = pickle.load(f)

    # Create simulator
    sim = Run(season, 
              gp, 
              data_tables, 
              driver_strategies)

    sim.run()

    # Store simulator with dynamic name (and in dict)
    globals()[cfg["var_name"]] = sim.outcomes
    Probabilistic_simulators[cfg["var_name"]] = sim.outcomes
    print(f"Simulation completed for: {gp}")
    print("=" * 60)


 Running Simulation for: Austin
Simulation completed for: Austin
 Running Simulation for: Suzuka
Simulation completed for: Suzuka
 Running Simulation for: MexicoCity
Simulation completed for: MexicoCity
 Running Simulation for: YasMarina
Simulation completed for: YasMarina


## Run Deterministic Simulation

In [6]:
import pickle
from run import Run

# === Dictionary to store simulators by name ===
simulators = {}

# === Loop through each GP ===
for gp, cfg in gp_configs.items():
    print("=" * 60)
    print(f" Running Simulation for: {gp}")
    

    # Load strategy
    with open(f"../data/{cfg['strategy_file']}", "rb") as f:
        driver_strategies = pickle.load(f)

    # Create simulator
    sim = Run(season, 
              gp, 
              data_tables, 
              driver_strategies,
              test=True)

    sim.run()

    # Store simulator with dynamic name (and in dict)
    globals()[cfg["var_name"]] = sim.outcomes
    simulators[cfg["var_name"]] = sim.outcomes
    print(f"Simulation completed for: {gp}")
    print("=" * 60)


🏁 Running Simulation for: Austin
Simulation completed for: Austin
🏁 Running Simulation for: Suzuka
Simulation completed for: Suzuka
🏁 Running Simulation for: MexicoCity
Simulation completed for: MexicoCity
🏁 Running Simulation for: YasMarina
Simulation completed for: YasMarina
